# Agentic Website Analysis Demo
## Multi-Agent System for Marketing Team Autonomy

This demo showcases a working multi-agent system that analyzes websites and provides actionable design recommendations for Marketing teams.

### Demo Flow:
1. **Web Acquisition Agent** - Fetches and analyzes website structure
2. **Analysis Agent** - AI-powered analysis of design patterns and UX issues  
3. **Design Suggestion Agent** - Generates Marketing-focused improvement recommendations

In [33]:
import os
import multiprocessing
from playwright.sync_api import sync_playwright
import requests
from urllib.parse import urljoin, urlparse
import json
from dotenv import load_dotenv
from anthropic import Anthropic
import re
from datetime import datetime

# Load environment variables
load_dotenv()

# Initialize Anthropic client
try:
    client = Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))
except Exception as e:
    print(f"⚠️  Anthropic client initialization failed: {e}")
    client = None

In [34]:
## 🤖 Agent 1: Web Acquisition Agent (Auto-Port Selection)

import shutil
from pathlib import Path
import urllib.parse
import threading
import subprocess
import concurrent.futures
import time
import socket

class WebAcquisitionAgent:
    """Web Acquisition Agent with HTML structure preservation - Thread isolated solution"""
    
    def __init__(self):
        self.base_dir = Path('../sites')
        self.base_dir.mkdir(exist_ok=True)

    def fetch_website(self, url):
        """Fetch website with full HTML structure and assets for local editing"""
        print(f"🔍 Web Acquisition Agent: Capturing {url} with full structure")
        
        # Create site-specific directory
        site_name = url.replace('https://', '').replace('http://', '').replace('www.', '').replace('/', '_')
        site_dir = self.base_dir / site_name
        site_dir.mkdir(exist_ok=True)
        
        try:
            # Run Playwright in separate thread to completely avoid asyncio conflicts
            with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
                future = executor.submit(self._scrape_in_thread, url, str(site_dir))
                result = future.result(timeout=120)  # 2 minute timeout
            
            if 'error' in result:
                print(f"❌ Error fetching website: {result['error']}")
                return None
            
            # Save the complete HTML file
            html_path = site_dir / 'index.html'
            with open(html_path, 'w', encoding='utf-8') as f:
                f.write(result['html_content'])
            
            # Save analysis data
            analysis_path = site_dir / 'analysis.json'
            with open(analysis_path, 'w', encoding='utf-8') as f:
                json.dump(result, f, indent=2, ensure_ascii=False)
            
            print(f"✅ Website captured with structure at: {site_dir}")
            print(f"   📄 HTML: {html_path}")
            print(f"   📊 Analysis: {analysis_path}")
            print(f"   🎨 Assets: {len(result['assets']['css'])} CSS, {len(result['assets']['js'])} JS, {len(result['assets']['images'])} images")
            
            return {
                'analysis': result,
                'html_path': str(html_path),
                'site_dir': str(site_dir),
                'assets': result['assets']
            }
            
        except Exception as e:
            print(f"❌ Error in website capture: {str(e)}")
            return None

    def _scrape_in_thread(self, url, output_dir):
        """Run Playwright scraping in completely isolated thread"""
        try:
            # Import and run sync Playwright in thread (no asyncio involvement)
            from playwright.sync_api import sync_playwright
            
            with sync_playwright() as p:
                browser = p.chromium.launch(headless=True)
                page = browser.new_page()
                
                print(f"   🌐 Navigating to {url}")
                # Navigate and wait for React content to load
                page.goto(url, wait_until='networkidle', timeout=60000)
                page.wait_for_timeout(3000)  # Extra time for React rendering
                
                print(f"   📄 Extracting HTML content")
                # Get the complete rendered HTML
                html_content = page.content()
                
                print(f"   🎨 Downloading assets")
                # Download all assets (CSS, JS, images)
                assets = self._download_all_assets_sync(page, url, output_dir)
                
                # Modify HTML to use local assets
                modified_html = self._localize_html_assets(html_content, assets, url)
                
                print(f"   📊 Analyzing page structure")
                # Extract analysis data
                analysis = {
                    'url': url,
                    'title': page.title(),
                    'html_content': modified_html,
                    'original_html': html_content,
                    'assets': assets,
                    'headings': self._extract_headings_sync(page),
                    'navigation': self._extract_navigation_sync(page),
                    'calls_to_action': self._extract_ctas_sync(page),
                    'forms': self._extract_forms_sync(page),
                    'images': self._extract_images_data_sync(page),
                    'content_sections': self._extract_content_sections_sync(page),
                    'meta_description': self._extract_meta_description_sync(page),
                    'timestamp': datetime.now().isoformat()
                }
                
                browser.close()
                print(f"   ✅ Thread scraping complete")
                return analysis
                
        except Exception as e:
            print(f"   ❌ Thread scraping error: {str(e)}")
            return {'error': str(e), 'url': url}

    def _download_all_assets_sync(self, page, base_url, output_dir):
        """Download all CSS, JS, and image assets synchronously"""
        assets = {'css': [], 'js': [], 'images': []}
        
        # Create asset directories
        css_dir = Path(output_dir) / 'css'
        js_dir = Path(output_dir) / 'js'
        img_dir = Path(output_dir) / 'images'
        
        css_dir.mkdir(parents=True, exist_ok=True)
        js_dir.mkdir(parents=True, exist_ok=True)
        img_dir.mkdir(parents=True, exist_ok=True)
        
        # Download CSS files
        css_elements = page.query_selector_all('link[rel="stylesheet"]')
        for i, element in enumerate(css_elements):
            href = element.get_attribute('href')
            if href:
                full_url = urljoin(base_url, href)
                local_filename = f'style_{i}.css'
                if self._download_asset(full_url, css_dir / local_filename):
                    assets['css'].append({
                        'original_url': full_url,
                        'local_path': f'css/{local_filename}',
                        'href': href
                    })
        
        # Download JS files (limit to avoid too many files)
        js_elements = page.query_selector_all('script[src]')
        for i, element in enumerate(js_elements[:10]):  # Limit JS files
            src = element.get_attribute('src')
            if src and not src.startswith('data:'):
                full_url = urljoin(base_url, src)
                local_filename = f'script_{i}.js'
                if self._download_asset(full_url, js_dir / local_filename):
                    assets['js'].append({
                        'original_url': full_url,
                        'local_path': f'js/{local_filename}',
                        'src': src
                    })
        
        # Download images (limit to avoid too many files)
        img_elements = page.query_selector_all('img')
        for i, element in enumerate(img_elements[:15]):  # Limit images
            src = element.get_attribute('src')
            if src and not src.startswith('data:'):
                full_url = urljoin(base_url, src)
                extension = Path(urllib.parse.urlparse(full_url).path).suffix or '.jpg'
                local_filename = f'image_{i}{extension}'
                if self._download_asset(full_url, img_dir / local_filename):
                    assets['images'].append({
                        'original_url': full_url,
                        'local_path': f'images/{local_filename}',
                        'src': src
                    })
        
        return assets

    def _download_asset(self, url, local_path):
        """Download a single asset file"""
        try:
            response = requests.get(url, timeout=10, headers={
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
            })
            response.raise_for_status()
            
            with open(local_path, 'wb') as f:
                f.write(response.content)
            
            return True
        except Exception as e:
            # Don't print every failed download to reduce noise
            return False

    def _localize_html_assets(self, html_content, assets, base_url):
        """Modify HTML to use local asset paths"""
        modified_html = html_content
        
        # Replace CSS links
        for css_asset in assets['css']:
            modified_html = modified_html.replace(
                f'href="{css_asset["href"]}"',
                f'href="{css_asset["local_path"]}"'
            )
        
        # Replace JS sources
        for js_asset in assets['js']:
            modified_html = modified_html.replace(
                f'src="{js_asset["src"]}"',
                f'src="{js_asset["local_path"]}"'
            )
        
        # Replace image sources
        for img_asset in assets['images']:
            modified_html = modified_html.replace(
                f'src="{img_asset["src"]}"',
                f'src="{img_asset["local_path"]}"'
            )
        
        return modified_html

    def _extract_headings_sync(self, page):
        """Extract all headings with hierarchy"""
        headings = []
        for level in range(1, 7):
            elements = page.query_selector_all(f'h{level}')
            for element in elements:
                text = element.text_content()
                if text and text.strip():
                    headings.append({
                        'level': level,
                        'text': text.strip(),
                        'tag': f'h{level}'
                    })
        return headings

    def _extract_navigation_sync(self, page):
        """Extract navigation elements"""
        nav_items = []
        selectors = ['nav a', 'header a', '.nav a', '.navigation a', '[role="navigation"] a']
        
        for selector in selectors:
            elements = page.query_selector_all(selector)
            for element in elements:
                text = element.text_content()
                href = element.get_attribute('href')
                if text and text.strip() and len(text.strip()) < 100:
                    nav_items.append({'text': text.strip(), 'href': href})
        
        return nav_items[:20]

    def _extract_ctas_sync(self, page):
        """Extract call-to-action elements - FIXED"""
        ctas = []
        selectors = [
            'button', 'a[class*="button"]', 'a[class*="btn"]', 
            'input[type="submit"]', '.cta', '[class*="call-to-action"]'
        ]
        
        for selector in selectors:
            elements = page.query_selector_all(selector)
            for element in elements:
                text = element.text_content()
                href = element.get_attribute('href')
                # Fix: Use evaluate to get tag name instead of tag_name property
                tag_name = element.evaluate('el => el.tagName.toLowerCase()')
                if text and text.strip() and len(text.strip()) < 200:
                    ctas.append({
                        'text': text.strip(),
                        'type': tag_name,
                        'href': href
                    })
        
        return ctas[:15]

    def _extract_forms_sync(self, page):
        """Extract form elements"""
        forms = []
        form_elements = page.query_selector_all('form')
        
        for form in form_elements:
            inputs = form.query_selector_all('input, textarea, select')
            form_data = {
                'action': form.get_attribute('action'),
                'method': form.get_attribute('method') or 'GET',
                'inputs': []
            }
            
            for input_elem in inputs:
                form_data['inputs'].append({
                    'type': input_elem.get_attribute('type'),
                    'name': input_elem.get_attribute('name'),
                    'placeholder': input_elem.get_attribute('placeholder')
                })
            
            forms.append(form_data)
        
        return forms

    def _extract_images_data_sync(self, page):
        """Extract image information"""
        images = []
        img_elements = page.query_selector_all('img')
        
        for img in img_elements[:10]:  # Limit to first 10
            src = img.get_attribute('src')
            alt = img.get_attribute('alt')
            if src:
                images.append({
                    'src': src,
                    'alt': alt,
                    'loading': img.get_attribute('loading')
                })
        
        return images

    def _extract_content_sections_sync(self, page):
        """Extract main content sections"""
        sections = []
        section_selectors = ['main', 'section', 'article', '.content', '#content']
        
        for selector in section_selectors:
            elements = page.query_selector_all(selector)
            for element in elements:
                text = element.text_content()
                if text and text.strip() and len(text.strip()) > 50:
                    sections.append({
                        'selector': selector,
                        'text_length': len(text),
                        'preview': text[:200] + '...' if len(text) > 200 else text
                    })
        
        return sections

    def _extract_meta_description_sync(self, page):
        """Extract meta description"""
        meta_elem = page.query_selector('meta[name="description"]')
        return meta_elem.get_attribute('content') if meta_elem else ''

    def _find_free_port(self, start_port=8000):
        """Find an available port starting from start_port"""
        for port in range(start_port, start_port + 100):
            try:
                with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                    s.bind(('localhost', port))
                    return port
            except OSError:
                continue
        return None

    def start_local_server(self, site_dir, start_port=8000):
        """Start local server for preview with automatic port selection"""
        
        # Find available port
        port = self._find_free_port(start_port)
        if not port:
            print("❌ No available ports found")
            return None
            
        def run_server():
            try:
                print(f"🌐 Starting server on port {port}")
                subprocess.run([
                    'python', '-m', 'http.server', str(port)
                ], cwd=site_dir, check=True)
            except subprocess.CalledProcessError as e:
                print(f"❌ Server error: {e}")
            except KeyboardInterrupt:
                print(f"🛑 Server on port {port} stopped")
        
        server_thread = threading.Thread(target=run_server, daemon=True)
        server_thread.start()
        
        # Give server time to start
        time.sleep(1)
        
        server_url = f"http://localhost:{port}"
        print(f"🌐 Local server started at {server_url}")
        print(f"📁 Serving files from: {site_dir}")
        return server_url

# Initialize the Web Acquisition Agent
web_agent = WebAcquisitionAgent()
print("🤖 Web Acquisition Agent (Auto-Port + Fixed) initialized!")

# Note about React limitations
print("\n💡 Note: Static server works for captured React content!")
print("   ✅ Visual layout preserved (perfect for Marketing edits)")
print("   ❌ Interactive features disabled (buttons, forms won't work)")
print("   🎯 Ideal for HTML/CSS modifications and design changes")

🤖 Web Acquisition Agent (Auto-Port + Fixed) initialized!

💡 Note: Static server works for captured React content!
   ✅ Visual layout preserved (perfect for Marketing edits)
   ❌ Interactive features disabled (buttons, forms won't work)
   🎯 Ideal for HTML/CSS modifications and design changes


In [35]:
## Test Complete Website Capture with Structure Preservation

# Test the enhanced Web Acquisition Agent
url = 'https://sportdev.us/'
print(f"🧪 Testing complete website capture: {url}")

# Capture website with full structure
capture_result = web_agent.fetch_website(url)

if capture_result:
    print("\n📊 Capture Results:")
    print(f"   📄 HTML saved to: {capture_result['html_path']}")
    print(f"   📁 Site directory: {capture_result['site_dir']}")
    
    # Show analysis summary
    analysis = capture_result['analysis']
    print(f"\n📈 Analysis Summary:")
    print(f"   Title: {analysis.get('title', 'N/A')}")
    print(f"   Headings: {len(analysis.get('headings', []))} found")
    print(f"   Navigation: {len(analysis.get('navigation', []))} items")
    print(f"   CTAs: {len(analysis.get('calls_to_action', []))} found")
    print(f"   Forms: {len(analysis.get('forms', []))} found")
    print(f"   Images: {len(analysis.get('images', []))} found")
    
    # Show assets downloaded
    assets = capture_result['assets']
    print(f"\n🎨 Assets Downloaded:")
    print(f"   CSS files: {len(assets['css'])}")
    print(f"   JS files: {len(assets['js'])}")
    print(f"   Images: {len(assets['images'])}")
    
    # Start local server for preview
    print(f"\n🌐 Starting local preview server...")
    server_url = web_agent.start_local_server(capture_result['site_dir'])
    
    print(f"\n✅ Complete website capture successful!")
    print(f"🔗 View at: {server_url}")
    print(f"📝 Marketing can now edit: {capture_result['html_path']}")
    
else:
    print("❌ Failed to capture website structure.")

🧪 Testing complete website capture: https://sportdev.us/
🔍 Web Acquisition Agent: Capturing https://sportdev.us/ with full structure
   🌐 Navigating to https://sportdev.us/
   📄 Extracting HTML content
   🎨 Downloading assets
   📊 Analyzing page structure
   ✅ Thread scraping complete
✅ Website captured with structure at: ../sites/sportdev.us_
   📄 HTML: ../sites/sportdev.us_/index.html
   📊 Analysis: ../sites/sportdev.us_/analysis.json
   🎨 Assets: 6 CSS, 9 JS, 9 images

📊 Capture Results:
   📄 HTML saved to: ../sites/sportdev.us_/index.html
   📁 Site directory: ../sites/sportdev.us_

📈 Analysis Summary:
   Title: Sport.Dev
   Headings: 15 found
   Navigation: 4 items
   CTAs: 1 found
   Forms: 0 found
   Images: 10 found

🎨 Assets Downloaded:
   CSS files: 6
   JS files: 9
   Images: 9

🌐 Starting local preview server...
🌐 Starting server on port 8000
Serving HTTP on :: port 8000 (http://[::]:8000/) ...
🌐 Local server started at http://localhost:8000
📁 Serving files from: ../sites/sp

::1 - - [04/Jul/2025 01:14:12] "GET / HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /css/style_0.css HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /css/style_1.css HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /css/style_3.css HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /css/style_4.css HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /js/script_1.js HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /css/style_2.css HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /css/style_5.css HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /js/script_3.js HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /js/script_2.js HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /js/script_4.js HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /js/script_5.js HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /js/script_6.js HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /js/script_0.js HTTP/1.1" 200 -
::1 - - [04/Jul/2025 01:14:12] "GET /images/image_0.png 

In [36]:
## 🧠 Agent 2: Analysis Agent

class AnalysisAgent:
    """AI-powered analysis of website design patterns and UX issues"""
    
    def __init__(self, anthropic_client):
        self.client = anthropic_client
    
    def analyze_website(self, website_data):
        """Analyze website structure and identify design patterns and issues"""
        print("🧠 Analysis Agent: Processing website data...")
        
        # Prepare structured data for AI analysis
        analysis_prompt = f"""
        You are a senior UX/UI analyst reviewing a website for a Marketing team. 
        Analyze the following website data and provide insights on:
        1. Overall design patterns and user experience
        2. Marketing effectiveness (messaging, CTAs, user journey)
        3. Accessibility and usability concerns
        4. Content hierarchy and information architecture
        
        Website Data:
        Title: {website_data['title']}
        Meta Description: {website_data['meta_description']}
        
        Headings Structure:
        {json.dumps(website_data['headings'], indent=2)}
        
        Navigation Items: {website_data['navigation'][:10]}
        
        Call-to-Action Buttons: {website_data['calls_to_action']}
        
        Forms Present: {len(website_data['forms'])} forms detected
        
        Images: {len(website_data['images'])} images found
        
        Content Sections: {len(website_data['content_sections'])} main content areas
        
        Provide a structured analysis in the following format:
        
        DESIGN PATTERNS:
        - [Key design patterns observed]
        
        MARKETING EFFECTIVENESS:
        - [Assessment of messaging and conversion elements]
        
        USER EXPERIENCE ISSUES:
        - [Potential usability problems]
        
        ACCESSIBILITY CONCERNS:
        - [Basic accessibility observations]
        
        INFORMATION ARCHITECTURE:
        - [Content organization and hierarchy assessment]
        
        Keep your analysis concise but actionable for a Marketing team.
        """
        
        try:
            response = self.client.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=1000,
                temperature=0.3,
                messages=[
                    {"role": "user", "content": analysis_prompt}
                ]
            )
            
            analysis_result = response.content[0].text
            print("✅ Website analysis complete!")
            return analysis_result
            
        except Exception as e:
            print(f"❌ Error in analysis: {str(e)}")
            return f"Analysis failed: {str(e)}"

# Initialize the Analysis Agent  
analysis_agent = AnalysisAgent(client)
print("🧠 Analysis Agent initialized!")

🧠 Analysis Agent initialized!


In [37]:
## 💡 Agent 3: Design Modification Agent (HTML Structure Editing)

class DesignModificationAgent:
    """Generates and applies specific HTML/CSS modifications for Marketing team"""
    
    def __init__(self, anthropic_client):
        self.client = anthropic_client
    
    def generate_modifications(self, website_data, analysis_result, modification_request):
        """Generate specific HTML/CSS modifications based on Marketing request"""
        print("💡 Design Modification Agent: Creating specific edits...")
        
        modification_prompt = f"""
        You are a Marketing-focused web designer. Based on the website analysis and Marketing request,
        generate specific HTML/CSS modifications that can be applied to the existing website structure.
        
        Website Analysis:
        {analysis_result}
        
        Current Website Structure:
        - Title: {website_data['title']}
        - Headings: {len(website_data.get('headings', []))} found
        - CTAs: {len(website_data.get('calls_to_action', []))} found
        - Navigation: {len(website_data.get('navigation', []))} items
        
        Marketing Request: {modification_request}
        
        Provide specific modifications in this format:
        
        🎯 RECOMMENDED MODIFICATIONS:
        
        1. [MODIFICATION TYPE] - [SPECIFIC CHANGE]
           Target: [CSS selector or HTML element]
           Change: [Exact CSS or HTML modification]
           Reason: [Why this improves marketing effectiveness]
           
        2. [Continue for 2-3 modifications]
        
        🔧 CSS MODIFICATIONS:
        ```css
        /* Specific CSS rules to add/modify */
        ```
        
        📝 HTML MODIFICATIONS:
        ```html
        <!-- Specific HTML changes -->
        ```
        
        💡 IMPLEMENTATION NOTES:
        - [Step-by-step instructions for Marketing team]
        
        Keep modifications focused on high-impact marketing improvements.
        """
        
        try:
            response = self.client.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=1500,
                temperature=0.3,
                messages=[
                    {"role": "user", "content": modification_prompt}
                ]
            )
            
            modifications = response.content[0].text
            print("✅ Design modifications generated!")
            return modifications
            
        except Exception as e:
            print(f"❌ Error generating modifications: {str(e)}")
            return f"Modification generation failed: {str(e)}"

    def apply_modifications(self, html_path, modifications_text):
        """Apply generated modifications to the HTML file"""
        print("🔧 Design Modification Agent: Applying changes...")
        
        try:
            # Read current HTML
            with open(html_path, 'r', encoding='utf-8') as f:
                html_content = f.read()
            
            # Create backup
            backup_path = html_path.replace('.html', '_backup.html')
            with open(backup_path, 'w', encoding='utf-8') as f:
                f.write(html_content)
            
            # Extract CSS modifications from the AI response
            css_pattern = r'```css\n(.*?)\n```'
            css_matches = re.findall(css_pattern, modifications_text, re.DOTALL)
            
            # Extract HTML modifications from the AI response  
            html_pattern = r'```html\n(.*?)\n```'
            html_matches = re.findall(html_pattern, modifications_text, re.DOTALL)
            
            modified_html = html_content
            
            # Apply CSS modifications
            if css_matches:
                css_modifications = '\n'.join(css_matches)
                # Add CSS to the head section
                if '</head>' in modified_html:
                    css_block = f'\n<style>\n/* Marketing Modifications */\n{css_modifications}\n</style>\n'
                    modified_html = modified_html.replace('</head>', css_block + '</head>')
                else:
                    # Add CSS at the beginning of the document
                    css_block = f'<style>\n/* Marketing Modifications */\n{css_modifications}\n</style>\n'
                    modified_html = css_block + modified_html
            
            # Apply HTML modifications (this would need more sophisticated parsing)
            # For now, we'll just add the modifications as comments for Marketing team review
            if html_matches:
                html_modifications = '\n'.join(html_matches)
                modification_comment = f'\n<!-- MARKETING MODIFICATIONS TO APPLY:\n{html_modifications}\n-->\n'
                modified_html = modification_comment + modified_html
            
            # Save modified HTML
            with open(html_path, 'w', encoding='utf-8') as f:
                f.write(modified_html)
            
            print(f"✅ Modifications applied!")
            print(f"   📄 Modified: {html_path}")
            print(f"   💾 Backup: {backup_path}")
            
            return {
                'success': True,
                'backup_path': backup_path,
                'css_applied': len(css_matches) > 0,
                'html_notes': len(html_matches) > 0
            }
            
        except Exception as e:
            print(f"❌ Error applying modifications: {str(e)}")
            return {'success': False, 'error': str(e)}

# Initialize the Design Modification Agent
modification_agent = DesignModificationAgent(client)
print("💡 Design Modification Agent initialized!")

💡 Design Modification Agent initialized!


## 📈 Demo Value Proposition

**What the team just witnessed:**

✅ **Multi-Agent System**: 3 specialized AI agents working in coordination  
✅ **Real Website Analysis**: Live scraping and analysis of any website  
✅ **AI-Powered Insights**: Advanced analysis using Claude 3.5 Sonnet  
✅ **Marketing-Focused Output**: Actionable recommendations for Marketing teams  
✅ **30-Second Execution**: Rapid analysis that scales to any website  

**Immediate Business Value:**
- **Time Savings**: Replace hours of manual analysis with 30-second automated insights
- **Marketing Autonomy**: Enable Marketing team to analyze competitor sites independently  
- **Consistent Analysis**: Standardized evaluation framework across all websites
- **Actionable Output**: Specific recommendations, not just generic observations
- **Scalability**: Analyze hundreds of websites with the same effort as one

**Next Steps:**
1. Expand to local website rendering and modification
2. Add A/B testing and design option generation  
3. Build user-friendly interface for non-technical users
4. Integrate with existing Marketing tools and workflows

*This demo represents Day 1 of the 2-week development sprint. The foundation is solid and ready for rapid expansion!*

In [ ]:
# 🚀 LIVE DEMO - Try It Now!

# Demo Configuration
DEMO_URLS = [
    "https://www.familysearch.org/en/campaign/temple-ord-ready",  # Simple, reliable test site
    "https://anthropic.com",  # AI company with modern design
    "https://github.com",  # Popular developer platform
    "https://stripe.com",  # Clean, conversion-focused design
]

print("🚀 AGENTIC WEBSITE ANALYSIS DEMO")
print("=" * 50)
print("Available demo URLs:")
for i, url in enumerate(DEMO_URLS, 1):
    print(f"  {i}. {url}")
print("\nTo run demo after all agents are initialized:")
print("  orchestrator.run_complete_analysis('URL_HERE')")
print("=" * 50)

# Quick demo function for easier testing
def quick_demo(url_index=1):
    """Run demo with one of the predefined URLs"""
    if url_index < 1 or url_index > len(DEMO_URLS):
        print(f"❌ Invalid URL index. Choose 1-{len(DEMO_URLS)}")
        return
    
    url = DEMO_URLS[url_index - 1]
    try:
        return orchestrator.run_complete_analysis(url)
    except NameError:
        print("❌ Error: Please run all agent initialization cells first!")
        print("   Required execution order:")
        print("   1. Setup cell (imports and dependencies)")
        print("   2. Web Acquisition Agent")
        print("   3. Analysis Agent") 
        print("   4. Design Suggestion Agent")
        print("   5. Demo Orchestrator")
        print("   6. Then you can run demos!")
        return None

print("💡 Quick demo function available: quick_demo(1) through quick_demo(4)")

🚀 AGENTIC WEBSITE ANALYSIS DEMO
Available demo URLs:
  1. https://www.familysearch.org/en/campaign/temple-ord-ready
  2. https://anthropic.com
  3. https://github.com
  4. https://stripe.com

To run demo after all agents are initialized:
  orchestrator.run_complete_analysis('URL_HERE')
💡 Quick demo function available: quick_demo(1) through quick_demo(4)
